<a href="https://colab.research.google.com/github/wangqianyang/colab/blob/main/%E8%8B%B1%E8%AF%AD%E9%9F%B3%E9%A2%91%E7%94%9F%E6%88%90%E5%AD%97%E5%B9%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 步骤一：生成SRT字幕

In [ ]:
# 安装 faster-whisper（更快、更省显存）和 ffmpeg
!pip install -q faster-whisper
!apt-get install -y ffmpeg -qq

### 上传文件

In [ ]:
from google.colab import files

# 上传你的音频文件（支持 mp3, wav, m4a, mp4 等常见格式）
uploaded = files.upload()

# 获取上传的文件名（假设只上传一个音频文件）
audio_file = list(uploaded.keys())[0]
print(f"已上传音频文件：{audio_file}")

### 加载模型并转录，生成 SRT 字幕

In [ ]:
from faster_whisper import WhisperModel
import datetime
import os

# 模型设置（如果 large-v3 显存不够，改成 "medium"）
model_size = "large-v3"  # 试试 "medium" 或 "small" 如果报错

model = WhisperModel(model_size, device="cuda", compute_type="float16")

# 关键修改：关闭 VAD 过滤 + 自动检测语言 + 降低阈值
segments_iterator, info = model.transcribe(
    audio_file,
    language=None,          # 自动检测语言（很重要！）
    beam_size=5,
    vad_filter=False,       # 先关闭 VAD，避免过度过滤
    vad_parameters=dict(min_silence_duration_ms=500),  # 即使开 VAD 也放宽
    word_timestamps=True,
    temperature=0.0,        # 更确定性输出
    no_speech_threshold=0.6,  # 默认 0.6，调低到 0.4 可以更敏感（如果仍空再试）
)

# 将 segments 迭代器转换为列表，以便多次使用
segments = list(segments_iterator)

# === 诊断信息输出 ===
print(f"检测到语言: {info.language} (概率: {info.language_probability:.2f})")
print(f"转录时长: {info.duration:.2f} 秒")
print(f"检测到段落数量: {len(segments)}") # 使用已转换为列表的 segments

if info.language_probability < 0.5:
    print("警告：语言检测置信度低，可能是非语音内容或噪音。")

if len(segments) == 0: # 使用已转换为列表的 segments
    print("错误：没有检测到任何语音段落！可能原因：")
    print("  - 音频无声或只有背景音乐")
    print("  - 文件损坏")
    print("  - 需要更低的 no_speech_threshold（试试 0.4）")
    print("  - 尝试关闭 vad_filter 或换小模型")
else:
    print("成功检测到语音！")

# 如果有段落，才生成 SRT
if len(segments) > 0: # 使用已转换为列表的 segments
    def format_timestamp(seconds: float):
        td = datetime.timedelta(seconds=seconds)
        hours, remainder = divmod(td.total_seconds(), 3600)
        minutes, seconds = divmod(remainder, 60)
        milliseconds = int((seconds - int(seconds)) * 1000)
        return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"

    srt_content = ""
    for i, segment in enumerate(segments, start=1): # 使用已转换为列表的 segments
        start_time = format_timestamp(segment.start)
        end_time = format_timestamp(segment.end)
        text = segment.text.strip()
        srt_content += f"{i}\n{start_time} --> {end_time}\n{text}\n\n"

    srt_filename = os.path.splitext(audio_file)[0] + ".srt"
    with open(srt_filename, "w", encoding="utf-8") as f:
        f.write(srt_content)

    print(f"SRT 字幕已保存为：{srt_filename}")
    files.download(srt_filename)
else:
    print("由于没有检测到语音，SRT 文件未生成。")

# 始终预览诊断（即使为空）
print("\n=== 额外调试：尝试打印原始 info ===")
print(info)


### （可选）：预览前几行字幕

In [ ]:
# 打印前 10 段字幕预览
print("\n=== 前 10 段字幕预览 ===\n")
preview_segments = list(segments)[:10]
for i, segment in enumerate(preview_segments, start=1):
    print(f"{i}\n{format_timestamp(segment.start)} --> {format_timestamp(segment.end)}\n{segment.text.strip()}\n")

## 步骤二：AI识别重点并生成双语+注解

In [ ]:
# Cell 1: 安装必要库（只需运行一次）
!pip install -q openai pysrt

In [ ]:
# Cell 2: 上传或加载你的 SRT 文件（如果已经上传过，可跳过上传直接指定文件名）
from google.colab import files
import pysrt
import os

# 如果你已经运行过第一步，SRT 文件应该已经在 Colab 环境中
# 直接指定文件名（替换成你的实际 SRT 文件名，例如 "your_audio.srt"）
srt_filename = "001.srt"  # <--- 这里改成你的 SRT 文件名 !!!

# 如果文件不在，直接上传
if not os.path.exists(srt_filename):
    print("文件不存在，请上传 SRT 文件")
    uploaded = files.upload()
    srt_filename = list(uploaded.keys())[0]

# 加载 SRT 并提取纯文本句子（按段落顺序）
subs = pysrt.open(srt_filename, encoding='utf-8')

# 提取每段的纯文本（去掉时间戳，只保留文字）
sentences = [sub.text.strip().replace('\n', ' ') for sub in subs if sub.text.strip()]

print(f"成功加载 SRT，共 {len(sentences)} 段句子")
print("\n前 10 句预览：")
for i, sent in enumerate(sentences[:10], 1):
    print(f"{i}: {sent}")

### 使用 OpenRouter GPT-OSS 模型 识别重点单词和词组

#### 1. 设置 OpenRouter API Key

您需要一个 OpenRouter API 密钥。如果还没有，请前往 [https://openrouter.ai/keys](https://openrouter.ai/keys) 创建一个。
在 Colab 中，通过左侧面板的“🔑”图标将密钥添加到 Secrets Manager。将其命名为 `OPENROUTER_API_KEY`。然后代码将自动获取该密钥。


In [ ]:
import getpass
from google.colab import userdata

# 获取 Colab Secrets 中保存的 OPENROUTER_API_KEY
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

if not OPENROUTER_API_KEY:
    print("警告：未找到 OPENROUTER_API_KEY。请确保您已在 Colab Secrets 中设置它。")
else:
    print("OpenRouter API Key 已设置")


In [ ]:
# Cell 4: 定义 Prompt 模板 + 批量调用 OpenRouter API
import json
import time
from openai import OpenAI # OpenRouter 兼容 OpenAI API

# 使用 OpenAI 兼容格式调用 OpenRouter API
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# OpenRouter 模型名称
model_name = "openai/gpt-oss-120b:free"

# Prompt 模板（严格参考 BBC / EnglishClass101 风格）
prompt_template = """
你是英语学习字幕专家，参考BBC Learning English、English Addict with Mr Steve和EnglishClass101的字幕风格。
为每句英文输出：
1. 英文原文：重点单词/短语用[HL:yellow]标记（生词/高频新词），习语/搭配用[HL:red]标记。
2. 中文翻译：自然流畅、适合学习者。
3. 注解行（可选）：仅对重点词添加简短括号解释，如 (run into = 偶遇，meet by chance)。注解要极简，只出现必要时。

输出严格为 JSON 数组（无需其他文字），每个对象对应一句：
[
  {{
    "english": "标记后的英文",
    "chinese": "中文翻译",
    "annotation": "注解（如果没有则为空字符串 ""）"
  }},
  ...
]

现在处理以下句子（每句用数字编号）：
{numbered_sentences}

只输出 JSON！
"""

# 批量大小（建议 8-12 句一批，避免超过 token 限制）
batch_size = 10

enhanced_data = []

print("开始批量调用 OpenRouter API 处理...")
for i in range(0, len(sentences), batch_size):
    batch = sentences[i:i+batch_size]

    # 编号显示
    numbered = "\n".join(f"{j+1}: {sent}" for j, sent in enumerate(batch))

    prompt = prompt_template.format(numbered_sentences=numbered)

    print(f"处理第 {i+1}-{min(i+batch_size, len(sentences))} 句（共 {len(batch)} 句）...")

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,      # 低温度更稳定
            max_tokens=4096
        )

        content = response.choices[0].message.content.strip()

        # 尝试解析 JSON
        try:
            batch_json = json.loads(content)
        except json.JSONDecodeError:
            print("JSON 解析失败，原始输出：")
            print(content)
            print("请手动修正或重新运行本批")
            continue

        enhanced_data.extend(batch_json)

    except Exception as e:
        print(f"API 调用出错：{e}")
        print("可能原因：API Key 无效、配额不足、网络问题")
        break

    # 礼貌等待，避免触发限流
    time.sleep(2)

print(f"\n处理完成！共增强 {len(enhanced_data)} 段")


In [ ]:
# Cell 5: 保存增强 JSON 并下载
import json
from google.colab import files

json_filename = os.path.splitext(srt_filename)[0] + "_enhanced.json"

with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(enhanced_data, f, ensure_ascii=False, indent=2)

print(f"增强 JSON 已保存为：{json_filename}")
print("\n前 3 条预览：")
print(json.dumps(enhanced_data[:3], ensure_ascii=False, indent=2))

# 自动下载
files.download(json_filename)

## 步骤三：自动生成ASS字幕（全自动，Python脚本）

In [ ]:
import pysrt
import json
import re

# 加载原SRT（时间轴）
subs = pysrt.open(srt_filename)

# 加载AI处理的JSON
with open(json_filename, 'r', encoding='utf-8') as f:
    enhanced = json.load(f)

# ASS颜色定义
colors = {
    'yellow': '&H00FFFF&',   # ASS黄色
    'orange': '&H00A5FF&',
    'red': '&H0000FF&',
    'white': '&HFFFFFF&'
}

def add_color(text):
    # 替换[HL:color]为ASS标签
    def repl(match):
        word = match.group(1)
        color = colors.get(match.group(2), colors['white'])
        return f'{{\\c{color}}}{word}{{\\c{colors["white"]}}}'
    return re.sub(r'\[HL:(\w+?)\](.*?)\[/HL\]', repl, text)

# 合并
for i, sub in enumerate(subs):
    item = enhanced[i]
    english = add_color(item['english'])
    chinese = item['chinese']
    annotation = item.get('annotation', '')
    # ASS三行布局
    sub.text = f"{english}\\N{{\\fs20}}{chinese}\\N{{\\fs18\\c&H808080&}}{annotation}"

# 保存ASS
with open('final.ass', 'w', encoding='utf-8') as f:
    f.write('[Script Info]\nScriptType: v4.00+\n\n[V4+ Styles]\nFormat: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\nStyle: Default,Arial,28,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,30,1\n\n[Events]\nFormat: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text\n')
    for sub in subs:
        f.write(f'Dialogue: 0,{sub.start},{sub.end},Default,,0,0,0,,{sub.text}\n')

print("ASS字幕生成完成！")

## 【调试用，不执行】下载youtube的音频试试

In [ ]:
# 安装 yt-dlp
!pip install -q yt-dlp


In [ ]:
import yt_dlp
import os

def download_youtube_audio(youtube_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(title)s.%(ext)s', # Output filename template
        'quiet': True, # Suppress console output for cleaner execution
        'no_warnings': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(youtube_url, download=True)
            filename = ydl.prepare_filename(info_dict)
            # Change extension to mp3 explicitly
            base, ext = os.path.splitext(filename)
            mp3_filename = base + '.mp3'
            print(f"成功下载音频：{mp3_filename}")
            return mp3_filename
    except Exception as e:
        print(f"下载失败：{e}")
        return None

# 示例用法：
# youtube_url = 'YOUR_YOUTUBE_URL_HERE'
# downloaded_file = download_youtube_audio(youtube_url)
# if downloaded_file:
#     print(f"文件已保存到：{downloaded_file}")


In [ ]:
youtube_url = 'https://www.youtube.com/watch?v=f96t2rsb4Lw'
downloaded_file = download_youtube_audio(youtube_url)
if downloaded_file:
    print(f"文件已保存到：{downloaded_file}")

现在你可以使用上面的 `download_youtube_audio` 函数来下载YouTube视频的音频。只需将你想要下载的YouTube链接替换掉 `YOUR_YOUTUBE_URL_HERE` 即可。

## 测试gemini模型

In [ ]:
# 安装 Google 官方最新的 GenAI SDK (支持 Gemini 2.0/3.0 系列)
!pip install -q -U google-genai

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata
import textwrap
from IPython.display import display, Markdown

# ---------------------------------------------------------
# 1. 初始化客户端
# ---------------------------------------------------------
# 建议在 Colab 左侧 "Secrets" 中设置 GOOGLE_API_KEY
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
except Exception:
    API_KEY = "你的_API_KEY_填在这里"

client = genai.Client(api_key=API_KEY)

# ---------------------------------------------------------
# 2. 模型配置 (重点：设置思考能力)
# ---------------------------------------------------------
# Gemini 3 Pro Preview 的核心特性是可控的思考深度
generate_config = types.GenerateContentConfig(
    temperature=0.7,
    top_p=0.95,
    max_output_tokens=8192,

    # 【关键设置】启用深度思考模式
    # 'HIGH' 适合复杂逻辑/编程，'LOW' 适合快速对话
    # 注意：这是 Gemini 3 / 2.0 Flash Thinking 系列特有的参数结构
    thinking_config=types.ThinkingConfig(
        include_thoughts=True # 让模型返回它的思考过程
    )
)

# ---------------------------------------------------------
# 3. 定义提示词
# ---------------------------------------------------------
prompt = """
请分析一下：如果人类实现了可控核聚变，对全球地缘政治格局会有哪些具体的、分阶段的影响？
请先进行深度的思维链推导，然后再给出最终结论。
"""

# ---------------------------------------------------------
# 4. 调用模型 (使用流式传输 Stream)
# ---------------------------------------------------------
# 这里的模型 ID 是我们在上文中讨论的预览版 ID
MODEL_ID = "gemini-3-pro-preview"

# 如果你是 2024/2025 年的用户，请取消下面这行的注释以使用当时可用的模型：
# MODEL_ID = "gemini-2.0-flash-thinking-exp"

print(f"正在调用模型: {MODEL_ID} ...\n")

try:
    response = client.models.generate_content_stream(
        model=MODEL_ID,
        contents=prompt,
        config=generate_config
    )

    # ---------------------------------------------------------
    # 5. 处理并显示流式输出
    # ---------------------------------------------------------
    final_text = ""
    print("--- 回答开始 ---\n")

    for chunk in response:
        # 检查是否有文本内容
        if chunk.text:
            print(chunk.text, end="", flush=True)
            final_text += chunk.text

    # 如果开启了 include_thoughts，通常 API 会在元数据或特定字段返回思考过程
    # 这里演示简单的文本接收

except Exception as e:
    print(f"\n\n调用出错: {e}")
    print("提示：请检查你的 API Key 是否有权限访问该预览版模型。")

## 声音提取字幕

In [ ]:
# 安装 openai-whisper
!pip install git+https://github.com/openai/whisper.git -q

# 安装 ffmpeg (音频处理必须)
!sudo apt update && sudo apt install ffmpeg -q

print("安装完成！请继续下一步。")

In [ ]:
import os
from google.colab import files

# 清理旧文件（可选）
!rm -f *.mp3 *.wav *.m4a *.mp4

print("请上传需要提取字幕的音频/视频文件：")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

print(f"已上传文件: {file_name}")

请上传需要提取字幕的音频/视频文件：


Saving veronikaslanguagediaries.mp3 to veronikaslanguagediaries.mp3
已上传文件: veronikaslanguagediaries.mp3


In [ ]:
# 为了处理文件名中的空格，我们对文件名加引号
import shlex

safe_filename = shlex.quote(file_name)

print("正在提取字幕，请耐心等待（取决于文件时长）...")
# 运行 Whisper
# 参数说明：
# --model large-v3 : 使用最新的大模型以获得最高精度
# --output_format srt : 仅输出 srt 字幕文件
# --verbose False : 减少输出日志，只看结果
!whisper {safe_filename} --model large-v3 --output_format srt --verbose False

print("提取完成！")

正在提取字幕，请耐心等待（取决于文件时长）...
Detected language: English
100% 59346/59346 [03:55<00:00, 251.60frames/s]
提取完成！


In [ ]:
# 获取生成的 srt 文件名（Whisper 会生成与原文件名同名的 .srt 文件）
base_name = os.path.splitext(file_name)[0]
srt_file = base_name + ".srt"

if os.path.exists(srt_file):
    print(f"找到字幕文件: {srt_file}\n")

    # 打印前10行预览
    print("=== 字幕预览 (前10行) ===")
    with open(srt_file, 'r', encoding='utf-8') as f:
        print("".join(f.readlines()[:10]))
    print("========================\n")

    # 自动下载
    files.download(srt_file)
else:
    print("未找到字幕文件，请检查上一步是否报错。")

## 使用Qwen3克隆声音

In [ ]:
# 安装 Qwen-TTS 核心库及相关依赖
!pip install -U qwen-tts transformers accelerate librosa soundfile
# 为了更快的推理速度（可选，T4 GPU 编译较慢）
# !pip install flash-attn --no-build-isolation

In [ ]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel
from google.colab import files
from IPython.display import Audio, display

# 1. 模型配置
# 也可以换成 Qwen/Qwen3-TTS-12Hz-1.7B-Instruct (如果有对话版)
model_id = "Qwen/Qwen3-TTS-12Hz-1.7B-Base"

print(f"正在从 Hugging Face 加载模型: {model_id}...")

# 2. 加载模型
# 提示：T4 GPU 显存约为 15GB，1.7B 模型在 bfloat16 下约占用 4-5GB
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = Qwen3TTSModel.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.bfloat16,
    # 如果没安装 flash-attn，请保持使用 "sdpa"
    attn_implementation="sdpa"
)

# 3. 上传参考音频 (Voice Prompt)
print("\n--- 请上传参考音频文件 (建议 3-10 秒，WAV/MP3) ---")
uploaded = files.upload()
if not uploaded:
    raise ValueError("未上传任何文件，请重新运行此单元格。")

ref_audio_path = list(uploaded.keys())[0]

# 4. 设置参数
# 注意：提供参考音频中的准确文本有助于模型更好地捕捉音色
ref_text = input("请输入参考音频中正在说的内容（支持中/英/日等）: ")
target_text = input("请输入你想要合成的目标文本: ")

print("\n--- 正在执行声音克隆 ---")

# 5. 执行推理
# 第一步：提取音频特征作为 Prompt
voice_clone_prompt = model.create_voice_clone_prompt(
    ref_audio=ref_audio_path,
    ref_text=ref_text
)

# 第二步：根据 Prompt 生成目标语音
# 该模型支持中、英、日、韩、德、法、意、俄、西、葡
wavs, sr = model.generate_voice_clone(
    text=target_text,
    language="english", # 这里可以根据需要修改目标语言
    voice_clone_prompt=voice_clone_prompt
)

# 6. 保存并播放结果
output_filename = "huggingface_output.wav"
sf.write(output_filename, wavs[0], sr)

print(f"\n生成成功！已保存为: {output_filename}")
display(Audio(output_filename, autoplay=True))

## 使用Qwen3设计声音再生成音频

In [ ]:
!pip install -U qwen-tts transformers accelerate soundfile

In [ ]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel
from IPython.display import Audio, display

# 1. 加载模型
model_id = "Qwen/Qwen3-TTS-12Hz-1.7B-Base"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"正在加载 Qwen3-TTS 模型...")
model = Qwen3TTSModel.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa" # Colab T4 建议使用 sdpa
)

# 2. 语音设计 (Voice Design)
# 这是关键步骤：通过文字描述定义音色
print("\n--- 正在通过描述设计老者音色 ---")

# 描述词建议使用英文以获得更精确的理解
# 描述包含：性别、年龄、音调、语速、性格特点
voice_description = (
    "A deep-voiced elderly American man. "
    "His voice is low-pitched, slightly gravelly, sounding very wise and calm. "
    "He speaks at a very slow and deliberate pace, with clear pauses."
)

# 给设计好的声音一段“初始文本”作为参考
design_ref_text = "Knowledge is proud that he has learned so much; Wisdom is humble that he knows no more."

# 生成虚拟的音色 Prompt
voice_design_prompt = model.generate_voice_design(
    description=voice_description,
    ref_text=design_ref_text
)

# 3. 设置要朗读的哲理文章
# 我们增加了一些标点符号（如 ...）来引导模型停顿更久
philosophical_article = """
The Starfish Story.
An old man was walking on the beach after a storm...
He saw a young boy picking up starfish and throwing them back into the ocean.
The old man asked: Why are you doing this? You cannot save them all. It does not matter.
The boy picked up one more starfish... and threw it into the water.
He smiled and said: It matters... to this one.
"""

# 4. 执行克隆合成 (Voice Clone)
print("正在使用设计的音色生成语音...")
wavs, sr = model.generate_voice_clone(
    text=philosophical_article,
    language="English",
    voice_clone_prompt=voice_design_prompt
)

# 5. 保存并展示
output_path = "designed_old_man_voice.wav"
sf.write(output_path, wavs[0], sr)

print(f"\n生成成功！已保存为: {output_path}")
display(Audio(output_path, autoplay=True))